# 2024 누적선박밀집도와 해양쓰레기, 해류 데이터를 함께 보기


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib.quiver import Quiver
import matplotlib.font_manager as fm
import matplotlib as mpl

# 한글 폰트 설정 함수
def set_korean_font():
    """
    한글 폰트 설정을 위한 함수
    시스템에 설치된 한글 폰트를 찾아 설정
    """
    # 시스템 폰트 경로 리스트
    font_locations = [
        # Windows
        'C:/Windows/Fonts/malgun.ttf',                # 맑은 고딕
        'C:/Windows/Fonts/gulim.ttc',                 # 굴림
        'C:/Windows/Fonts/batang.ttc',                # 바탕
        'C:/Windows/Fonts/NanumGothic.ttf',           # 나눔 고딕
        'C:/Windows/Fonts/NanumBarunGothic.ttf',      # 나눔 바른 고딕
        
        # macOS
        '/Library/Fonts/AppleGothic.ttf',             # Apple 고딕
        '/Library/Fonts/NanumGothic.ttf',             # 나눔 고딕
        
        # Linux
        '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',  # 나눔 고딕 (Ubuntu)
        '/usr/share/fonts/nanum/NanumGothic.ttf',           # 나눔 고딕 (CentOS)
    ]
    
    # 사용 가능한 폰트 확인
    font_found = False
    
    for font_path in font_locations:
        if os.path.exists(font_path):
            print(f"한글 폰트를 찾았습니다: {font_path}")
            # 폰트 설정
            plt.rcParams['font.family'] = 'sans-serif'
            plt.rcParams['font.sans-serif'] = [fm.FontProperties(fname=font_path).get_name(), 'DejaVu Sans']
            font_found = True
            break
    
    if not font_found:
        print("한글 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
        # 폰트가 없을 경우 기본 설정
        plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
    
    # 유니코드 마이너스 문제 해결
    mpl.rcParams['axes.unicode_minus'] = False
    
    return font_found

def integrated_analysis(base_path, station_grid_file, debris_data_file, current_data_file, 
                        output_file='integrated_analysis.png', num_days=30):
    """
    선박밀집도, 해류, 해양쓰레기를 통합 분석하는 함수
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - debris_data_file: 해양쓰레기 데이터 파일 경로
    - current_data_file: 해류 데이터 파일 경로
    - output_file: 출력 이미지 파일 경로
    - num_days: 누적할 일수 (기본 30일)
    """
    # 한글 폰트 설정
    set_korean_font()
    
    print("="*50)
    print("선박밀집도, 해류, 해양쓰레기 통합 분석을 시작합니다.")
    print("="*50)
    
    # 1. 해양쓰레기 데이터 로드
    print("\n1. 해양쓰레기 데이터 로드")
    debris_data = {}
    if os.path.exists(debris_data_file):
        debris_df = pd.read_csv(debris_data_file)
        
        # 개수 또는 무게 정보가 있는지 확인
        if '개수' in debris_df.columns:
            # 정점별 쓰레기 개수 저장
            for _, row in debris_df.iterrows():
                debris_data[row['정점명']] = {
                    '개수': row['개수'],
                    '무게_kg': row['무게_kg'] if '무게_kg' in debris_df.columns else 0
                }
            
            # 개수 범위 설정
            min_debris = debris_df['개수'].min()
            max_debris = debris_df['개수'].max()
            
            print(f"- 해양쓰레기 데이터 로드: 개수 범위 {min_debris}~{max_debris}")
            print(f"- 총 {len(debris_data)}개의 정점 데이터가 로드되었습니다.")
        else:
            print("- 해양쓰레기 데이터에 '개수' 컬럼이 없습니다.")
    else:
        print(f"- 경고: 해양쓰레기 데이터 파일이 존재하지 않습니다: {debris_data_file}")
        return
    
    # 2. 정점별 가장 가까운 그리드 정보 로드
    print("\n2. 정점별 가장 가까운 그리드 정보 로드")
    station_grids = {}
    station_locations = {}
    
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        for _, row in station_grid_data.iterrows():
            station_grids[row['정점명']] = row['가장_가까운_그리드']
            
            # 정점 위치 정보도 저장 (있는 경우)
            if '위도' in station_grid_data.columns and '경도' in station_grid_data.columns:
                station_locations[row['정점명']] = (row['위도'], row['경도'])
                
        print(f"- 해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"- 경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
        return
    
    # 3. 해류 데이터 로드
    print("\n3. 해류 데이터 로드")
    if os.path.exists(current_data_file):
        current_df = pd.read_csv(current_data_file)
        
        # datetime 문자열을 datetime 객체로 변환
        current_df['datetime'] = pd.to_datetime(current_df['datetime'])
        
        print(f"- 해류 데이터 로드: 총 {len(current_df)}개 데이터 포인트")
        print(f"- 데이터 기간: {current_df['datetime'].min()} ~ {current_df['datetime'].max()}")
        
        # 해류 방향(current_dir)과 속도(current_speed) 확인
        print(f"- 해류 방향 범위: {current_df['current_dir'].min()}° ~ {current_df['current_dir'].max()}°")
        print(f"- 해류 속도 범위: {current_df['current_speed'].min()} ~ {current_df['current_speed'].max()} 단위")
        
        # 사용 가능한 위경도 범위 확인
        lon_range = (current_df['pre_lon'].min(), current_df['pre_lon'].max())
        lat_range = (current_df['pre_lat'].min(), current_df['pre_lat'].max())
        print(f"- 위경도 범위: 경도 {lon_range}, 위도 {lat_range}")
    else:
        print(f"- 경고: 해류 데이터 파일이 존재하지 않습니다: {current_data_file}")
        current_df = None
    
    # 4. 선박밀집도 데이터 누적 계산
    print("\n4. 선박밀집도 데이터 누적 계산")
    
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 모든 시간대 포맷 (00시부터 23시까지)
    hours = [f"{h:02d}00" for h in range(24)]
    
    # 모든 파일 경로 수집
    file_paths = []
    
    for day in range(num_days):
        date = start_date + timedelta(days=day)
        date_formatted = date.strftime('%Y%m%d')
        
        for hour in hours:
            file_pattern = os.path.join(base_path, f"sden_{date_formatted}{hour}_grid3.csv")
            
            if os.path.exists(file_pattern):
                datetime_obj = datetime.strptime(f"{date_formatted}{hour}", '%Y%m%d%H%M')
                file_paths.append((datetime_obj, file_pattern))
    
    if not file_paths:
        print("- 선박밀집도 데이터 파일을 찾을 수 없습니다.")
        return
    
    # 시간 순으로 정렬
    file_paths.sort(key=lambda x: x[0])
    print(f"- 총 {len(file_paths)}개 시간대의 선박밀집도 데이터를 분석합니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    print(f"- 총 {len(grid_locations)}개의 그리드 위치 정보를 추출했습니다.")
    
    # 누적 선박밀집도 계산
    cumulative_density = {}
    
    print("- 선박밀집도 데이터 누적 계산 중...")
    for i, (datetime_obj, file_path) in enumerate(file_paths):
        if i % 24 == 0:  # 하루에 24개 시간대가 있으므로 진행상황 표시
            print(f"  처리 중: {datetime_obj.strftime('%Y-%m-%d %H:%M')} ({i+1}/{len(file_paths)})")
            
        try:
            data = pd.read_csv(file_path)
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    # 누적 선박밀집도 및 교통량 계산
                    if grid_id not in cumulative_density:
                        cumulative_density[grid_id] = {
                            '누적_밀집도': 0,
                            '누적_교통량': 0,
                            '데이터_수': 0
                        }
                    
                    cumulative_density[grid_id]['누적_밀집도'] += density
                    cumulative_density[grid_id]['누적_교통량'] += traffic
                    cumulative_density[grid_id]['데이터_수'] += 1
                except:
                    continue
        except Exception as e:
            print(f"  파일 처리 오류: {file_path}, 오류: {str(e)}")
    
    # 평균 선박밀집도 계산
    for grid_id, data in cumulative_density.items():
        if data['데이터_수'] > 0:
            data['평균_밀집도'] = data['누적_밀집도'] / data['데이터_수']
            data['평균_교통량'] = data['누적_교통량'] / data['데이터_수']
        else:
            data['평균_밀집도'] = 0
            data['평균_교통량'] = 0
    
    # 최대 누적 밀집도 찾기
    max_cumulative_density = max([data['누적_밀집도'] for data in cumulative_density.values()])
    print(f"- 최대 누적 선박밀집도: {max_cumulative_density:.2f}")
    
    # 5. 정점별 선박밀집도, 해류, 쓰레기 데이터 통합
    print("\n5. 정점별 선박밀집도, 해류, 쓰레기 데이터 통합")
    
    # 해양쓰레기 정점 통합 정보
    station_integrated_data = []
    
    for station, grid_id in station_grids.items():
        if station in debris_data and grid_id in cumulative_density:
            # 선박밀집도 정보
            density_data = cumulative_density[grid_id]
            cumulative_density_value = density_data['누적_밀집도']
            avg_density = density_data['평균_밀집도']
            cumulative_traffic = density_data['누적_교통량']
            avg_traffic = density_data['평균_교통량']
            
            # 위경도 정보
            if station in station_locations:
                lat, lon = station_locations[station]
            elif grid_id in grid_locations:
                lat, lon = grid_locations[grid_id]
            else:
                continue  # 위치 정보가 없으면 건너뜀
                
            # 해류 정보
            current_info = {
                'avg_speed': None,
                'avg_direction': None,
                'max_speed': None
            }
            
            if current_df is not None:
                # 가장 가까운 해류 데이터 찾기 (위경도 기준)
                # 실제 구현에서는 더 복잡한 로직이 필요할 수 있음
                # 예: 특정 반경 내의 모든 해류 데이터 평균 계산
                
                # 위경도 차이를 기준으로 가장 가까운 해류 데이터 포인트 찾기
                current_df['distance'] = np.sqrt(
                    (current_df['pre_lat'] - lat)**2 + 
                    (current_df['pre_lon'] - lon)**2
                )
                
                # 가장 가까운 10개 포인트 선택
                nearest_currents = current_df.nsmallest(10, 'distance')
                
                if not nearest_currents.empty:
                    current_info['avg_speed'] = nearest_currents['current_speed'].mean()
                    
                    # 방향은 원형 데이터이므로 평균 계산에 주의
                    # 단순화를 위해 산술 평균 사용
                    current_info['avg_direction'] = nearest_currents['current_dir'].mean()
                    current_info['max_speed'] = nearest_currents['current_speed'].max()
            
            # 데이터 통합
            station_integrated_data.append({
                '정점명': station,
                '위도': lat,
                '경도': lon,
                '쓰레기_개수': debris_data[station]['개수'],
                '쓰레기_무게_kg': debris_data[station]['무게_kg'],
                '누적_선박밀집도': cumulative_density_value,
                '평균_선박밀집도': avg_density,
                '누적_교통량': cumulative_traffic,
                '평균_교통량': avg_traffic,
                '해류_평균속도': current_info['avg_speed'],
                '해류_평균방향': current_info['avg_direction'],
                '해류_최대속도': current_info['max_speed']
            })
    
    if not station_integrated_data:
        print("- 통합 가능한 데이터가 없습니다.")
        return
    
    # 통합 데이터 DataFrame 생성
    integrated_df = pd.DataFrame(station_integrated_data)
    print(f"- 총 {len(integrated_df)}개 정점에 대해 통합 데이터를 생성했습니다.")
    
    # 상관관계 분석
    print("\n6. 상관관계 분석")
    
    # 쓰레기 양과 다른 변수들 사이의 상관관계 계산
    correlations = {}
    for column in ['누적_선박밀집도', '평균_선박밀집도', '누적_교통량', '평균_교통량']:
        if column in integrated_df.columns:
            corr = integrated_df['쓰레기_개수'].corr(integrated_df[column])
            correlations[column] = corr
            print(f"- {column}와(과) 쓰레기 개수의 상관계수: {corr:.3f}")
    
    # 해류 데이터가 있는 경우 해류 변수와의 상관관계도 계산
    if '해류_평균속도' in integrated_df.columns and integrated_df['해류_평균속도'].notna().any():
        for column in ['해류_평균속도', '해류_최대속도']:
            if column in integrated_df.columns:
                corr = integrated_df['쓰레기_개수'].corr(integrated_df[column])
                correlations[column] = corr
                print(f"- {column}와(과) 쓰레기 개수의 상관계수: {corr:.3f}")
    
    # 7. 통합 시각화
    print("\n7. 통합 시각화")
    
    # 큰 그림 생성 (2x2 그리드로 구성)
    fig = plt.figure(figsize=(20, 18))
    
    # 그리드 설정
    gs = fig.add_gridspec(2, 2, width_ratios=[1.5, 1], height_ratios=[1.5, 1])
    
    # 1. 메인 지도 (누적 선박밀집도 + 해양쓰레기 + 해류)
    ax_map = fig.add_subplot(gs[0, 0])
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 해양쓰레기 양에 따른 색상 정의 (파랑, 녹색, 빨강)
    debris_colors = {
        'low': (0, 0, 1),     # 파랑
        'medium': (0, 0.8, 0), # 녹색
        'high': (1, 0, 0)      # 빨강
    }
    
    # 1.1 선박밀집도 데이터 그리기
    x = []  # 경도
    y = []  # 위도
    s = []  # 마커 크기 (평균 교통량)
    c = []  # 색상 (누적 밀집도)
    
    for grid_id, data in cumulative_density.items():
        if grid_id in grid_locations:
            lat, lon = grid_locations[grid_id]
            y.append(lat)
            x.append(lon)
            s.append(max(data['평균_교통량'] / 5, 10))  # 평균 교통량에 따른 마커 크기 조정, 최소 크기 10
            c.append(data['누적_밀집도'])
    
    # 선박밀집도 산점도 그리기
    scatter = ax_map.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                          vmin=0, vmax=max_cumulative_density, edgecolors='k', linewidths=0.5)
    
    # 1.2 쓰레기 양 표시
    # 정점 데이터를 3등분하여 색상 결정
    if integrated_df['쓰레기_개수'].nunique() >= 3:
        # 표시되는 정점들의 쓰레기 양 목록 추출
        sorted_amounts = sorted(integrated_df['쓰레기_개수'])
        total_points = len(sorted_amounts)
        
        # 균등하게 3분할하는 임계값 설정
        third_index = total_points // 3
        two_thirds_index = 2 * third_index
        
        low_threshold = sorted_amounts[third_index] if third_index < total_points else min(sorted_amounts)
        high_threshold = sorted_amounts[two_thirds_index] if two_thirds_index < total_points else max(sorted_amounts)
    else:
        # 데이터 포인트가 적은 경우 균등 분할
        min_debris = integrated_df['쓰레기_개수'].min()
        max_debris = integrated_df['쓰레기_개수'].max()
        range_size = (max_debris - min_debris) / 3
        low_threshold = min_debris + range_size
        high_threshold = max_debris - range_size
    
    # 각 카테고리에 몇 개의 정점이 포함되는지 계산
    low_count = sum(1 for val in integrated_df['쓰레기_개수'] if val <= low_threshold)
    medium_count = sum(1 for val in integrated_df['쓰레기_개수'] if low_threshold < val <= high_threshold)
    high_count = sum(1 for val in integrated_df['쓰레기_개수'] if val > high_threshold)
    
    # 박스 크기 설정
    box_size = 0.08
    
    # 정점별 박스 색상 설정
    integrated_df['쓰레기_카테고리'] = pd.cut(
        integrated_df['쓰레기_개수'],
        bins=[0, low_threshold, high_threshold, float('inf')],
        labels=['low', 'medium', 'high']
    )
    
    # 정점 그리기
    for _, row in integrated_df.iterrows():
        # 쓰레기 양에 따라 색상 결정
        category = row['쓰레기_카테고리']
        color = debris_colors[category]
        face_color = (*color, 0.3)
        
        # 네모 그리기
        rect = patches.Rectangle(
            (row['경도'] - box_size/2, row['위도'] - box_size/2), box_size, box_size, 
            linewidth=2, edgecolor=color, facecolor=face_color
        )
        ax_map.add_patch(rect)
        
        # 정점 이름 표시
        ax_map.text(row['경도'], row['위도'] + box_size/2, row['정점명'], 
                 fontsize=8, color='black', ha='center', va='bottom',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    # 1.3 해류 벡터 표시 (시간에 따른 평균 해류)
    if current_df is not None:
        # 해류 데이터를 그리드화하여 표시
        grid_size = 0.5  # 그리드 크기 (도 단위)
        
        # 그리드 생성
        lon_grid = np.arange(np.floor(min(x)) - 1, np.ceil(max(x)) + 1, grid_size)
        lat_grid = np.arange(np.floor(min(y)) - 1, np.ceil(max(y)) + 1, grid_size)
        
        # 각 그리드 셀에 대한 해류 평균 계산
        current_grid = {}
        
        for i, lon in enumerate(lon_grid[:-1]):
            for j, lat in enumerate(lat_grid[:-1]):
                # 그리드 셀 범위
                lon_min, lon_max = lon, lon_grid[i + 1]
                lat_min, lat_max = lat, lat_grid[j + 1]
                
                # 그리드 셀 내의 해류 데이터 찾기
                cell_data = current_df[
                    (current_df['pre_lon'] >= lon_min) & (current_df['pre_lon'] < lon_max) &
                    (current_df['pre_lat'] >= lat_min) & (current_df['pre_lat'] < lat_max)
                ]
                
                if not cell_data.empty:
                    # 평균 해류 계산
                    avg_dir = cell_data['current_dir'].mean()
                    avg_speed = cell_data['current_speed'].mean()
                    
                    # 각도를 벡터 성분으로 변환 (방향은 해양학 표기법: 0도는 북쪽, 시계방향으로 증가)
                    u = -avg_speed * np.sin(np.radians(avg_dir))  # 동쪽 성분 (음수는 서쪽)
                    v = -avg_speed * np.cos(np.radians(avg_dir))  # 북쪽 성분 (음수는 남쪽)
                    
                    # 그리드 셀 중심 계산
                    cell_center = ((lon_min + lon_max) / 2, (lat_min + lat_max) / 2)
                    
                    current_grid[cell_center] = (u, v, avg_speed)
        
        # 해류 벡터 그리기
        for (center_lon, center_lat), (u, v, speed) in current_grid.items():
            # 화살표 길이 스케일 조정 (해류 속도에 비례)
            scale = 0.1 * speed / max(current_df['current_speed'].max(), 1)  # 최대 화살표 길이 조정
            
            # 화살표 색상 (해류 속도에 따라)
            arrow_color = plt.cm.viridis(speed / max(current_df['current_speed'].max(), 1))
            
            # 화살표 그리기
            ax_map.arrow(
                center_lon, center_lat, 
                u * scale, v * scale, 
                head_width=0.05, head_length=0.1, 
                fc=arrow_color, ec=arrow_color, alpha=0.7
            )
    
    # 범례 요소 생성
    low_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['low'], facecolor=(*debris_colors['low'], 0.3))
    med_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['medium'], facecolor=(*debris_colors['medium'], 0.3))
    high_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['high'], facecolor=(*debris_colors['high'], 0.3))
    
    # 화살표 범례 (해류)
    if current_df is not None:
        arrow_leg = ax_map.arrow(0, 0, 0.1, 0, head_width=0.05, head_length=0.1, fc='purple', ec='purple')
    
    # 범례 추가
    legend_elements = [
        low_patch, med_patch, high_patch
    ]
    
    legend_labels = [
        f'적은 쓰레기 (≤{low_threshold}개, {low_count}개 정점)', 
        f'중간 쓰레기 ({low_threshold}~{high_threshold}개, {medium_count}개 정점)', 
        f'많은 쓰레기 (>{high_threshold}개, {high_count}개 정점)'
    ]
    
    if current_df is not None:
        legend_elements.append(arrow_leg)
        legend_labels.append('해류 방향 및 세기')
    
    ax_map.legend(
        legend_elements, legend_labels,
        loc='upper left', fontsize=10
    )
    
    # 제목 및 레이블
    ax_map.set_title(f'{num_days}일간 누적 선박밀집도와 해양쓰레기 비교 ({start_date.strftime("%Y-%m-%d")}부터)', fontsize=14)
    ax_map.set_xlabel('경도', fontsize=12)
    ax_map.set_ylabel('위도', fontsize=12)
    
    # 컬러바 추가
    cbar = fig.colorbar(scatter, ax=ax_map)
    cbar.set_label('누적 선박밀집도')
    
    # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
    ax_map.set_xlim([126, 130])
    ax_map.set_ylim([34, 38])
    
    # 그리드 추가
    ax_map.grid(True, linestyle='--', alpha=0.6)
    
    # 상관관계 정보 추가
    corr_text = '\n'.join([f"{k}: {v:.3f}" for k, v in correlations.items()])
    ax_map.text(
        0.02, 0.05, 
        f'쓰레기 양과의 상관계수:\n{corr_text}',
        transform=ax_map.transAxes,
        fontsize=10,
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
    )
    
    # 2. 선박밀집도-쓰레기 양 산점도
    ax_scatter = fig.add_subplot(gs[0, 1])
    
    # 산점도 데이터 준비
    scatter_x = integrated_df['누적_선박밀집도']
    scatter_y = integrated_df['쓰레기_개수']
    
    # 색상 설정
    scatter_colors = [debris_colors[cat] for cat in integrated_df['쓰레기_카테고리']]
    
    # 산점도 그리기
    ax_scatter.scatter(scatter_x, scatter_y, c=scatter_colors, alpha=0.7, s=80)
    
    # 회귀선 추가
    if len(scatter_x) > 1:
        z = np.polyfit(scatter_x, scatter_y, 1)
        p = np.poly1d(z)
        ax_scatter.plot(sorted(scatter_x), p(sorted(scatter_x)), "r--", alpha=0.5)
        
        # 회귀식 텍스트 추가
        eq_text = f'y = {z[0]:.2f}x + {z[1]:.2f}, R² = {correlations["누적_선박밀집도"]**2:.2f}'
        ax_scatter.text(0.05, 0.95, eq_text, transform=ax_scatter.transAxes, 
                     fontsize=10, va='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    # 각 정점에 레이블 추가
    for i, row in integrated_df.iterrows():
        ax_scatter.annotate(row['정점명'], 
                         (row['누적_선박밀집도'], row['쓰레기_개수']),
                         fontsize=8, alpha=0.7,
                         xytext=(5, 5), textcoords='offset points')
    
    # 축 레이블 및 제목
    ax_scatter.set_xlabel('누적 선박밀집도', fontsize=12)
    ax_scatter.set_ylabel('쓰레기 개수', fontsize=12)
    ax_scatter.set_title('선박밀집도와 쓰레기 양의 관계', fontsize=14)
    ax_scatter.grid(True, linestyle='--', alpha=0.3)
    
    # 3. 해류 속도-쓰레기 양 산점도
    ax_current = fig.add_subplot(gs[1, 0])
    
    # 해류 데이터가 있는 경우에만 그리기
    if current_df is not None and '해류_평균속도' in integrated_df.columns and integrated_df['해류_평균속도'].notna().any():
        # 산점도 데이터 준비
        current_x = integrated_df['해류_평균속도']
        current_y = integrated_df['쓰레기_개수']
        
        # 산점도 그리기
        ax_current.scatter(current_x, current_y, c=scatter_colors, alpha=0.7, s=80)
        
        # 회귀선 추가
        if len(current_x) > 1 and not current_x.isna().all():
            valid_indices = ~current_x.isna() & ~current_y.isna()
            if sum(valid_indices) > 1:  # 최소 2개 이상의 유효한 데이터 포인트 필요
                valid_x = current_x[valid_indices]
                valid_y = current_y[valid_indices]
                
                z = np.polyfit(valid_x, valid_y, 1)
                p = np.poly1d(z)
                
                x_range = np.linspace(min(valid_x), max(valid_x), 100)
                ax_current.plot(x_range, p(x_range), "r--", alpha=0.5)
                
                # 상관계수 계산 및 회귀식 텍스트 추가
                corr_current = valid_x.corr(valid_y)
                eq_text = f'y = {z[0]:.2f}x + {z[1]:.2f}, R² = {corr_current**2:.2f}'
                ax_current.text(0.05, 0.95, eq_text, transform=ax_current.transAxes, 
                             fontsize=10, va='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        
        # 각 정점에 레이블 추가
        for i, row in integrated_df.iterrows():
            if pd.notna(row['해류_평균속도']):
                ax_current.annotate(row['정점명'], 
                                 (row['해류_평균속도'], row['쓰레기_개수']),
                                 fontsize=8, alpha=0.7,
                                 xytext=(5, 5), textcoords='offset points')
        
        # 축 레이블 및 제목
        ax_current.set_xlabel('해류 평균 속도', fontsize=12)
        ax_current.set_ylabel('쓰레기 개수', fontsize=12)
        ax_current.set_title('해류 속도와 쓰레기 양의 관계', fontsize=14)
        ax_current.grid(True, linestyle='--', alpha=0.3)
    else:
        ax_current.text(0.5, 0.5, '해류 데이터가 충분하지 않습니다', 
                     ha='center', va='center', fontsize=14)
        ax_current.axis('off')
    
    # 4. 다변량 분석 결과 표시
    ax_multi = fig.add_subplot(gs[1, 1])
    
    # 다변량 회귀 분석
    if len(integrated_df) > 3:  # 최소 데이터 포인트 필요
        try:
            # 입력 변수 선택 (누적_선박밀집도, 해류_평균속도)
            X_vars = ['누적_선박밀집도']
            
            if '해류_평균속도' in integrated_df.columns and integrated_df['해류_평균속도'].notna().any():
                X_vars.append('해류_평균속도')
            
            # 결측치 처리
            analysis_df = integrated_df.dropna(subset=X_vars + ['쓰레기_개수'])
            
            if len(analysis_df) > 3:  # 유효한 데이터가 충분한지 확인
                X = analysis_df[X_vars]
                y = analysis_df['쓰레기_개수']
                
                # 다변량 회귀 분석 (단순 구현)
                X = sm.add_constant(X)  # 상수항 추가
                model = sm.OLS(y, X).fit()
                
                # 결과 표시
                summary_text = f"다변량 회귀 분석 결과:\n\n"
                summary_text += f"R-squared: {model.rsquared:.3f}\n"
                summary_text += f"Adjusted R-squared: {model.rsquared_adj:.3f}\n\n"
                summary_text += "계수:\n"
                
                for i, var in enumerate(['상수항'] + X_vars):
                    summary_text += f"{var}: {model.params[i]:.4f} (p={model.pvalues[i]:.4f})\n"
                
                if len(X_vars) > 1:
                    # 상대적 중요도 계산
                    importance = abs(model.params[1:] * X.std()[1:])
                    importance = importance / importance.sum() * 100
                    
                    summary_text += "\n상대적 중요도 (%):\n"
                    for i, var in enumerate(X_vars):
                        summary_text += f"{var}: {importance[i]:.1f}%\n"
                
                # 텍스트 표시
                ax_multi.text(0.05, 0.95, summary_text, transform=ax_multi.transAxes,
                           fontsize=10, va='top', family='monospace')
                ax_multi.axis('off')
            else:
                ax_multi.text(0.5, 0.5, '다변량 분석을 위한 유효한 데이터가 부족합니다', 
                           ha='center', va='center', fontsize=12)
                ax_multi.axis('off')
        except Exception as e:
            print(f"다변량 분석 오류: {str(e)}")
            ax_multi.text(0.5, 0.5, f'다변량 분석 오류: {str(e)}', 
                       ha='center', va='center', fontsize=10, wrap=True)
            ax_multi.axis('off')
    else:
        ax_multi.text(0.5, 0.5, '다변량 분석을 위한 데이터가 부족합니다', 
                   ha='center', va='center', fontsize=14)
        ax_multi.axis('off')
    
    # 전체 제목
    fig.suptitle(f'선박밀집도, 해류, 해양쓰레기 통합 분석', fontsize=16, y=0.98)
    
    # 여백 조정
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    
    # 이미지 저장
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"통합 분석 결과 이미지를 {output_file}에 저장했습니다.")
    
    # CSV 파일로 통합 데이터 저장
    output_csv = output_file.replace('.png', '_data.csv')
    integrated_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"통합 분석 데이터를 {output_csv}에 저장했습니다.")
    
    plt.close()
    return integrated_df

# 추가 함수: 시간에 따른 해류 변화 분석
def analyze_current_time_series(current_data_file, output_file='current_timeseries.png'):
    """
    시간에 따른 해류 변화를 분석하고 시각화
    
    Parameters:
    - current_data_file: 해류 데이터 파일 경로
    - output_file: 출력 이미지 파일 경로
    """
    print("\n시간에 따른 해류 변화 분석")
    
    if not os.path.exists(current_data_file):
        print(f"경고: 해류 데이터 파일이 존재하지 않습니다: {current_data_file}")
        return
    
    # 한글 폰트 설정 (이미 메인에서 설정되었을 수 있지만 안전을 위해 재설정)
    set_korean_font()
    
    # 데이터 로드
    current_df = pd.read_csv(current_data_file)
    current_df['datetime'] = pd.to_datetime(current_df['datetime'])
    
    # 데이터 확인
    print(f"- 총 {len(current_df)}개의 해류 데이터 포인트")
    print(f"- 기간: {current_df['datetime'].min()} ~ {current_df['datetime'].max()}")
    
    # 특정 위치에서의 시계열 분석
    # 해안에 가까운 위치를 몇 개 선택
    coastal_points = []
    
    # 그리드 생성 후 해안에 가까운 포인트 선택
    lon_grid = np.linspace(125, 130, 10)
    lat_grid = np.linspace(33, 38, 10)
    
    for lon in lon_grid[0:3]:  # 서쪽 해안에 가까운 지점
        for lat in lat_grid:
            coastal_points.append((lon, lat))
    
    # 각 포인트에서 가장 가까운 해류 데이터 시계열 분석
    point_timeseries = {}
    
    for point_idx, (target_lon, target_lat) in enumerate(coastal_points[:5]):  # 최대 5개 포인트만 분석
        # 해당 포인트에서 가장 가까운 해류 데이터 찾기
        current_df['distance'] = np.sqrt(
            (current_df['pre_lon'] - target_lon)**2 + 
            (current_df['pre_lat'] - target_lat)**2
        )
        
        closest_idx = current_df['distance'].idxmin()
        closest_lon = current_df.loc[closest_idx, 'pre_lon']
        closest_lat = current_df.loc[closest_idx, 'pre_lat']
        
        # 선택된 위치의 모든 시간대 데이터 추출
        point_data = current_df[
            (np.isclose(current_df['pre_lon'], closest_lon, atol=0.01)) & 
            (np.isclose(current_df['pre_lat'], closest_lat, atol=0.01))
        ].copy()
        
        if len(point_data) > 0:
            point_data = point_data.sort_values('datetime')
            
            # 고유한 ID 생성
            point_id = f"Point_{point_idx+1}_({closest_lon:.2f}, {closest_lat:.2f})"
            point_timeseries[point_id] = point_data
    
    if not point_timeseries:
        print("- 시계열 분석을 위한 데이터가 충분하지 않습니다.")
        return
    
    # 시각화
    fig, axes = plt.subplots(len(point_timeseries), 2, figsize=(15, 5 * len(point_timeseries)))
    
    if len(point_timeseries) == 1:
        axes = [axes]
    
    for i, (point_id, data) in enumerate(point_timeseries.items()):
        # 해류 속도 시계열
        ax_speed = axes[i][0]
        ax_speed.plot(data['datetime'], data['current_speed'], 'b-', linewidth=2)
        ax_speed.set_title(f'{point_id} - 해류 속도 변화', fontsize=12)
        ax_speed.set_xlabel('날짜', fontsize=10)
        ax_speed.set_ylabel('해류 속도', fontsize=10)
        ax_speed.grid(True, linestyle='--', alpha=0.6)
        ax_speed.xaxis.set_major_formatter(DateFormatter('%m-%d'))
        plt.setp(ax_speed.xaxis.get_majorticklabels(), rotation=45)
        
        # 해류 방향 시계열
        ax_dir = axes[i][1]
        ax_dir.plot(data['datetime'], data['current_dir'], 'r-', linewidth=2)
        ax_dir.set_title(f'{point_id} - 해류 방향 변화', fontsize=12)
        ax_dir.set_xlabel('날짜', fontsize=10)
        ax_dir.set_ylabel('해류 방향 (도)', fontsize=10)
        ax_dir.grid(True, linestyle='--', alpha=0.6)
        ax_dir.xaxis.set_major_formatter(DateFormatter('%m-%d'))
        plt.setp(ax_dir.xaxis.get_majorticklabels(), rotation=45)
        
        # 방향은 0-360도 사이의 값이므로 y축 범위 설정
        ax_dir.set_ylim([0, 360])
        ax_dir.set_yticks(np.arange(0, 361, 45))
        ax_dir.set_yticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N'])
    
    # 전체 제목
    fig.suptitle('시간에 따른 해류 변화 분석', fontsize=16, y=0.98)
    
    # 여백 조정
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # 이미지 저장
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"해류 시계열 분석 결과를 {output_file}에 저장했습니다.")
    
    plt.close()
    return

# 메인 실행 코드
if __name__ == "__main__":
    # 한글 폰트 설정
    korean_font_available = set_korean_font()
    
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    debris_data_file = r'D:\marideb\code\marideb_location.csv'  # 해양쓰레기 데이터 파일
    current_data_file = r'D:\marideb\code\tidal_current_2024.csv'  # 해류 데이터 파일
    
    # 필요한 패키지 임포트
    import statsmodels.api as sm
    
    # 1. 통합 분석 수행
    integrated_df = integrated_analysis(
        base_path, station_grid_file, debris_data_file, current_data_file, 
        output_file='integrated_analysis_result.png', 
        num_days=30
    )
    
    # 2. 시간에 따른 해류 변화 분석
    analyze_current_time_series(current_data_file, output_file='current_timeseries_analysis.png')
    
    # 폰트 설정 알림
    if korean_font_available:
        print("\n한글 폰트가 성공적으로 적용되었습니다.")
    else:
        print("\n경고: 한글 폰트를 설정할 수 없어 일부 한글이 깨질 수 있습니다.")

한글 폰트를 찾았습니다: C:/Windows/Fonts/malgun.ttf
한글 폰트를 찾았습니다: C:/Windows/Fonts/malgun.ttf
선박밀집도, 해류, 해양쓰레기 통합 분석을 시작합니다.

1. 해양쓰레기 데이터 로드
- 해양쓰레기 데이터 로드: 개수 범위 45~11420
- 총 60개의 정점 데이터가 로드되었습니다.

2. 정점별 가장 가까운 그리드 정보 로드
- 해양쓰레기 정점 그리드 60개를 로드했습니다.

3. 해류 데이터 로드
- 해류 데이터 로드: 총 4532544개 데이터 포인트
- 데이터 기간: 2024-01-01 00:00:00 ~ 2024-12-31 23:00:00
- 해류 방향 범위: 0° ~ 360°
- 해류 속도 범위: 0.0 ~ 241.0 단위
- 위경도 범위: 경도 (125.00015, 129.86137), 위도 (33.04701, 37.69467)

4. 선박밀집도 데이터 누적 계산
- 총 715개 시간대의 선박밀집도 데이터를 분석합니다.
- 총 3765개의 그리드 위치 정보를 추출했습니다.
- 선박밀집도 데이터 누적 계산 중...
  처리 중: 2023-01-01 00:00 (1/715)
  처리 중: 2023-01-02 00:00 (25/715)
  처리 중: 2023-01-03 00:00 (49/715)
  처리 중: 2023-01-04 00:00 (73/715)
  처리 중: 2023-01-05 00:00 (97/715)
  처리 중: 2023-01-06 00:00 (121/715)
  처리 중: 2023-01-07 00:00 (145/715)
  처리 중: 2023-01-08 00:00 (169/715)
  처리 중: 2023-01-09 00:00 (193/715)
  처리 중: 2023-01-10 00:00 (217/715)
  처리 중: 2023-01-11 00:00 (241/715)
  처리 중: 2023-01-12 00:00 (265/715)
  처리 중: 2023-01-13 00:00 (289/7

C:\Users\TAN\AppData\Local\Temp\ipykernel_18092\1350961999.py:661: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary_text += f"{var}: {model.params[i]:.4f} (p={model.pvalues[i]:.4f})\n"
C:\Users\TAN\AppData\Local\Temp\ipykernel_18092\1350961999.py:661: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary_text += f"{var}: {model.params[i]:.4f} (p={model.pvalues[i]:.4f})\n"
C:\Users\TAN\AppData\Local\Temp\ipykernel_18092\1350961999.py:670: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavio

통합 분석 결과 이미지를 integrated_analysis_result.png에 저장했습니다.
통합 분석 데이터를 integrated_analysis_result_data.csv에 저장했습니다.

시간에 따른 해류 변화 분석
한글 폰트를 찾았습니다: C:/Windows/Fonts/malgun.ttf
- 총 4532544개의 해류 데이터 포인트
- 기간: 2024-01-01 00:00:00 ~ 2024-12-31 23:00:00
해류 시계열 분석 결과를 current_timeseries_analysis.png에 저장했습니다.

한글 폰트가 성공적으로 적용되었습니다.
